In [1]:
import os
import subprocess
from tqdm import tqdm
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from typing import Union
import numpy as np
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer


/home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
1+1

Task was destroyed but it is pending!
task: <Task pending name='Task-2' coro=<Kernel.poll_control_queue() running at /home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/site-packages/ipykernel/kernelbase.py:279> wait_for=<Future finished result=[<zmq.Frame(b'...c-b5c'...36B)>, <zmq.Frame(b'<IDS|MSG>')>, <zmq.Frame(b'...aa38b'...64B)>, <zmq.Frame(b'..."202'...189B)>, <zmq.Frame(b'{}')>, <zmq.Frame(b'{}')>, ...]> cb=[_chain_future.<locals>._call_set_state() at /home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/asyncio/futures.py:392]>


In [ ]:
1+1

# INIT

In [2]:
# Change according to hardware
DEVICE_1 = "cuda:0"
DEVICE_2 ="cuda:1"# "cpu"


In [3]:
torch.set_grad_enabled(False)

observer_name = "tiiuae/falcon-7b-instruct"
performer_name = "tiiuae/falcon-7b"

identical_tokens = (AutoTokenizer.from_pretrained(observer_name).vocab ==
                    AutoTokenizer.from_pretrained(performer_name).vocab)

identical_tokens


/home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


True

In [4]:
observer_model = AutoModelForCausalLM.from_pretrained(observer_name,
                                                                   device_map={"": DEVICE_1},
                                                                   trust_remote_code=True,
                                                                   torch_dtype=torch.bfloat16)

performer_model = AutoModelForCausalLM.from_pretrained(performer_name,
                                                                     device_map={"": DEVICE_2},
                                                                     trust_remote_code=True,
                                                                     torch_dtype=torch.bfloat16)

observer_model.eval()
performer_model.eval()

tokenizer = AutoTokenizer.from_pretrained(observer_name)




Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.31s/it]
/home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.14s/it]


In [5]:
def tokenize(text):
    return tokenizer(text, return_tensors="pt")

tokenize("Hello, my dog is cute")


{'input_ids': tensor([[9856,   23,  491, 3696,  304, 7209]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [6]:
@torch.inference_mode()
def get_logits(encodings):
    observer_logits = observer_model(**encodings.to(DEVICE_1)).logits
    performer_logits = performer_model(**encodings.to(DEVICE_2)).logits
    return observer_logits, performer_logits

encoding = tokenize('''Dr. Capy Cosmos, a capybara unlike any other, astounded the scientific community with his 
groundbreaking research in astrophysics. With his keen sense of observation and unparalleled ability to interpret 
cosmic data, he uncovered new insights into the mysteries of black holes and the origins of the universe. As he 
peered through telescopes with his large, round eyes, fellow researchers often remarked that it seemed as if the 
stars themselves whispered their secrets directly to him. Dr. Cosmos not only became a beacon of inspiration to 
aspiring scientists but also proved that intellect and innovation can be found in the most unexpected of creatures.'''[:100])


observer_logits, performer_logits = get_logits(encoding)
observer_logits, performer_logits


(tensor([[[-23.2500, -18.2500,  -9.6875,  ..., -10.6875, -12.2500,  -9.1875],
          [-13.7500, -19.6250, -14.1250,  ..., -14.9375, -16.6250, -10.9375],
          [-13.0625, -17.0000, -14.9375,  ..., -17.8750, -15.4375, -14.5000],
          ...,
          [-12.9375, -12.2500, -12.6875,  ..., -16.0000, -14.8750, -18.0000],
          [-16.8750, -15.5625, -16.2500,  ..., -16.7500, -18.3750, -15.8125],
          [-16.1250, -17.6250, -14.5000,  ..., -18.1250, -19.2500, -17.1250]]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[[-20.3750, -21.0000, -14.5625,  ..., -13.2500, -15.8750,  -8.5625],
          [-13.0625, -18.7500, -15.1875,  ..., -15.3125, -17.1250, -11.5625],
          [-12.5000, -17.6250, -16.6250,  ..., -17.3750, -16.2500, -14.3750],
          ...,
          [-10.8750, -12.3750, -11.9375,  ..., -14.1875, -14.6875, -16.6250],
          [-14.6250, -13.8750, -16.7500,  ..., -17.6250, -19.1250, -15.6250],
          [-13.6250, -15.5625, -14.5625,  ..., -18.5000, -19.0

In [7]:
S = observer_logits.shape[-2]
V = observer_logits.shape[-1]

observer_logits[..., :-1, :].contiguous().shape


torch.Size([1, 25, 65024])

In [8]:
encoding.input_ids[..., 1:].shape


torch.Size([1, 25])

In [9]:
loss = torch.nn.CrossEntropyLoss(reduction='none')

ppl = loss(observer_logits[..., :-1, :].contiguous().transpose(1, 2).to(DEVICE_2), 
     encoding.input_ids[..., 1:].contiguous().to(DEVICE_2)).float()

ppl, ppl.sum(1)


(tensor([[ 0.7148,  9.8125,  6.0000,  9.8125,  1.8281,  2.9688,  1.9297,  6.9375,
           0.0425,  0.0269, 11.5000,  0.2715,  0.1387,  0.3066,  9.0625,  1.7422,
           1.1250,  2.4531,  0.7188,  1.0938,  1.4297,  4.7188,  9.6250,  5.5938,
           8.8750]], device='cuda:1'),
 tensor([98.7275], device='cuda:1'))

In [10]:
softmax = torch.nn.Softmax(dim=-1)

performer_probs = softmax(performer_logits).view(-1, V)

performer_probs, performer_probs.shape


(tensor([[5.8265e-12, 3.1122e-12, 1.9500e-09,  ..., 7.2177e-09, 5.2387e-10,
          7.8604e-07],
         [7.5437e-08, 2.5648e-10, 9.0222e-09,  ..., 7.9744e-09, 1.3024e-09,
          3.3900e-07],
         [6.0536e-08, 3.6016e-10, 9.7498e-10,  ..., 4.6202e-10, 1.4188e-09,
          9.2550e-09],
         ...,
         [1.3877e-07, 3.0966e-08, 4.7963e-08,  ..., 5.0350e-09, 3.0559e-09,
          4.4020e-10],
         [2.1304e-08, 4.5169e-08, 2.5466e-09,  ..., 1.0623e-09, 2.3647e-10,
          7.8580e-09],
         [2.5332e-07, 3.6554e-08, 9.9186e-08,  ..., 1.9354e-09, 1.1714e-09,
          1.2573e-08]], device='cuda:1', dtype=torch.bfloat16),
 torch.Size([26, 65024]))

In [11]:
observer_scores = observer_logits.view(-1, V).to(DEVICE_2)
observer_scores, observer_scores.shape


(tensor([[-23.2500, -18.2500,  -9.6875,  ..., -10.6875, -12.2500,  -9.1875],
         [-13.7500, -19.6250, -14.1250,  ..., -14.9375, -16.6250, -10.9375],
         [-13.0625, -17.0000, -14.9375,  ..., -17.8750, -15.4375, -14.5000],
         ...,
         [-12.9375, -12.2500, -12.6875,  ..., -16.0000, -14.8750, -18.0000],
         [-16.8750, -15.5625, -16.2500,  ..., -16.7500, -18.3750, -15.8125],
         [-16.1250, -17.6250, -14.5000,  ..., -18.1250, -19.2500, -17.1250]],
        device='cuda:1', dtype=torch.bfloat16),
 torch.Size([26, 65024]))

In [12]:
xppl = loss(observer_scores[:-1], performer_probs[:-1]).view(-1, S - 1)

xppl, xppl.sum(1)


(tensor([[3.1406, 7.3750, 5.1875, 7.5000, 3.8906, 5.0312, 6.2188, 7.1250, 0.2793,
          0.2559, 4.4375, 1.2031, 0.9180, 1.6719, 4.5000, 1.8047, 4.0312, 4.6250,
          1.0156, 2.7656, 1.9453, 6.0000, 3.7812, 9.8125, 0.9180]],
        device='cuda:1', dtype=torch.bfloat16),
 tensor([95.5000], device='cuda:1', dtype=torch.bfloat16))

In [13]:
binocular_score = ppl.sum(1) / xppl.sum(1)

binocular_score


tensor([1.0338], device='cuda:1')

In [18]:
# redefine to handle batch of strings
def tokenize(batch):
    encodings = tokenizer(batch, return_tensors="pt", 
    padding="longest" if len(batch) > 1 else False, truncation=True,
    max_length=512, return_token_type_ids=False).to(DEVICE_1)
    return encodings

# redefinition with cuda sync
@torch.inference_mode()
def get_logits(encodings):
    observer_logits = observer_model(**encodings.to(DEVICE_1)).logits
    performer_logits = performer_model(**encodings.to(DEVICE_2)).logits
    torch.cuda.synchronize()

    return observer_logits, performer_logits

loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
softmax_fn = torch.nn.Softmax(dim=-1)

def perplexity(encoding, logits):
    shifted_logits = logits[..., :-1, :].contiguous()
    shifted_labels = encoding.input_ids[..., 1:].contiguous()
    shifted_attention_mask = encoding.attention_mask[..., 1:].contiguous()

    ppl = loss_fn(shifted_logits.transpose(1, 2).to(DEVICE_2), shifted_labels) * shifted_attention_mask
    ppl = ppl.sum(1) / shifted_attention_mask.sum(1)
    
    return ppl.to("cpu").float().numpy()

def cross_perplexity(observer_logits, performer_logits, encoding):
    V = observer_logits.shape[-1]
    S = observer_logits.shape[-2]

    performer_probs = softmax_fn(performer_logits).view(-1, V).to(DEVICE_2)
    observer_scores = observer_logits.view(-1, V).to(DEVICE_2)
    
    xppl = loss_fn(observer_scores, performer_probs).view(-1, S)
    padding_mask = (encoding.input_ids != tokenizer.pad_token_id).type(torch.uint8)
    
    xppl = (xppl * padding_mask).sum(1) / padding_mask.sum(1)
    
    return xppl.cpu().float().numpy()

def binocular_score(text):
    batch = [text] if isinstance(text, str) else text
    encodings = tokenize(batch)
    observer_logits, performer_logits = get_logits(encodings)
    ppl = perplexity(encodings, observer_logits)
    xppl = cross_perplexity(observer_logits, performer_logits, encodings)

    return (ppl / xppl).tolist()

tokenizer.pad_token = tokenizer.eos_token


In [24]:
tests = ['''The motivation behind LLM Detection is harm reduction, to trace text origins, block spam, and identify fake news produced by LLMs. *Preemptive detection* methods attempt to "watermark" generated text, but requires full control of the generating models, which already seems to be impossible. Therefore, more recent works have been on *post-hoc detection* methods, which could be used without the cooperation of the text's author. The paper's authors suggest that there are two main groups for post-hoc detectors, the first being finetuning a pretrained language model to perform binary classification. There are many additional techniques that make this approach more effective, but all implementations will require training on text produced by the target model, which is both computationally expensive and limited by the number of new models that are being open-sourced.
The second group uses statistical signatures of machine-generated text, with the aim of zero-shot learning. This would allow for the detection of a wide range of models, with little to no training data. These methods use measures such as perplexity, perplexity curvature, log rank, intrinsic dimensionality, and n-gram analysis. The Binoculars paper proposes a focus on low false positive rate (FPR) and high performance on out-of-domain samples, rather than focusing on classifier AUCs for the high-stakes application of LLM detection.''',
# '''Dr. Capy Cosmos, a capybara unlike any other, astounded the scientific community with his 
# groundbreaking research in astrophysics. With his keen sense of observation and unparalleled ability to interpret 
# cosmic data, he uncovered new insights into the mysteries of black holes and the origins of the universe. As he 
# peered through telescopes with his large, round eyes, fellow researchers often remarked that it seemed as if the 
# stars themselves whispered their secrets directly to him. Dr. Cosmos not only became a beacon of inspiration to 
# aspiring scientists but also proved that intellect and innovation can be found in the most unexpected of creatures.''',
# '''We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'''
]
binocular_score(tests)


[0.946601927280426]

In [ ]:
1+1

# OUT DATASET

In [20]:
import pandas as pd

# Load a parquet file into a pandas DataFrame
df_before = pd.read_parquet('chatgptbefore.parquet')

# Display the first few rows of the dataframe
print(df_before.head())


                                  unique_id  \
0  aed3pspeai9hu7magmpk5js3hlmd67ajr2804f01   
1  6qa9kcljo2dnsq2cffpm32uhhlmd2dhvcj940vg1   
2  hefbgudg1vehubvhg1f62omoto7rf21rem41k901   
3  m1anank010tp1fah6srogdmkso3rjdtb51rubmo1   
4  18rhnb3vdmv9un43arsblhl1r4s8adb452j382g1   

                                                body                 date  \
0                                                ...  2022-09-15 14:20:16   
1  Top Stories @PCWorld \r\n\r\nDon’t panic! Inte...  2022-06-15 19:06:11   
2  <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 T...  2022-11-25 20:26:28   
3  \r\n\n--- mail_boundary ---\n\r\n\r\n\r\n\r\n\...  2022-08-09 09:02:27   
4  \r\n\n--- mail_boundary ---\n\r\n\r\n\r\n\r\n\...  2022-06-07 19:17:58   

                                        cleaned_text  
0  p r x u l e ils sont de retour version en lign...  
1  top stories p c world dont panic intel says he...  
2  stradivarius vite profitezen profitez du black...  
3  mailboundary variety internatio

In [21]:
df_after = pd.read_parquet('chatgptafter.parquet')

# Display the first few rows of the dataframe
print(df_after.head())

                                  unique_id  \
0  24hpi0o3m4g5j3srjijhc5ivu08qmkqt4gkrn181   
1  f7jp9535uut6bcbljsu9ibdknpfu84vts1fsrso1   
2  grcaba458pvs9hh88hlfnuta9el10udo63k7kh01   
3  175s99re5e8tgupg6kjcl95l3nu4asshg9fcpsg1   
4  rine0d5tghjb9tuk06n1889jpsn9battpninei81   

                                                body                 date  
0  *Please consider adding <info@fsf.org> to your...  2023-02-22 06:17:47  
1  \r\n\n--- mail_boundary ---\n\r\n\r\n\r\n\r\n\...  2023-04-13 01:05:03  
2  Si vous n'arrivez pas à lire votre newsletter,...  2023-03-15 16:11:38  
3  No images?  Click here https://blackdoginstitu...  2023-01-16 05:59:06  
4  \r\nÉlégance artisanale\r\nhttps://link.email....  2023-04-24 06:04:46  


In [25]:
res_before_email=[]

tmp=list(df_before.values)
for elt in tqdm(tmp):
    date=elt[2]
    sentence=elt[1]

    proba = binocular_score(sentence)[0]
    
    res_before_email+=[[sentence,proba, date]]

100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [09:31<00:00,  2.63it/s]


In [26]:
res_after_email=[]

tmp=list(df_after.values)
for elt in tqdm(tmp):
    date=elt[2]
    sentence=elt[1]

    proba = binocular_score(sentence)[0]
    
    res_after_email+=[[sentence,proba, date]]

100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [09:29<00:00,  2.64it/s]


In [28]:
import json

In [29]:
f=open('results_scores_email_BINOCULAR.json','w')
json.dump({'res_after':res_after_email,'res_before':res_before_email},f)
f.close()

In [ ]:
res_before_para=[]

tmp=list(df_before.values)
for elt in tqdm(tmp):
    date=elt[2]
    for sentence in [x for x in elt[1].split('\n') if len(x)>3]:
        proba = binocular_score(sentence)[0]
 
        res_before_para+=[[sentence,proba, date]]

  0%|                                                                               | 1/1500 [00:51<21:36:29, 51.89s/it]

In [ ]:
res_after_para=[]

tmp=list(df_after.values)
for elt in tqdm(tmp):
    date=elt[2]
    for sentence in [x for x in elt[1].split('\n') if len(x)>3]:
        proba = binocular_score(sentence)[0]
     
        res_after_para+=[[sentence,proba, date]]

In [ ]:
f=open('results_scores_para_BINOCULAR.json','w')
json.dump({'res_after':res_after_para,'res_before':res_before_para},f)
f.close()

In [ ]:
res_before_sent=[]

tmp=list(df_before.values)
for elt in tqdm(tmp):
    date=elt[2]
    for sentence in [x for x in elt[1].replace('\n','').split('.') if len(x)>3]:
        proba = binocular_score(sentence)[0]
        res_before_sent+=[[sentence,proba, date]]

In [ ]:
res_after_sent=[]

tmp=list(df_after.values)
for elt in tqdm(tmp):
    date=elt[2]
    for sentence in [x for x in elt[1].replace('\n','').split('.') if len(x)>3]:
        proba = binocular_score(sentence)[0]
        res_after_sent+=[[sentence,proba, date]]

In [ ]:
import json

f=open('results_scores_sent_BINOCULAR.json','w')
json.dump({'res_after_sent':res_after_sent,'res_before_sent':res_before_sent},f)
f.close()

# EXTRA

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300

capybara = '''Dr. Capy Cosmos, a capybara unlike any other, astounded the scientific community with his 
groundbreaking research in astrophysics. With his keen sense of observation and unparalleled ability to interpret 
cosmic data, he uncovered new insights into the mysteries of black holes and the origins of the universe. As he 
peered through telescopes with his large, round eyes, fellow researchers often remarked that it seemed as if the 
stars themselves whispered their secrets directly to him. Dr. Cosmos not only became a beacon of inspiration to 
aspiring scientists but also proved that intellect and innovation can be found in the most unexpected of creatures.'''

encoding = tokenize([capybara])

observer_logits, performer_logits = get_logits(encoding)

S = observer_logits.shape[-2]
V = observer_logits.shape[-1]

(S, V)


In [ ]:
shifted_logits = observer_logits[..., :-1, :].contiguous()
shifted_labels = encoding.input_ids[..., 1:].contiguous()

ppl = loss_fn(shifted_logits.transpose(1, 2).to("cpu"), shifted_labels).float()

ppl, ppl.sum(1)


In [ ]:
from IPython.display import HTML

normalized_ppl = ppl / torch.max(ppl)

def generate_html(tokens, scores):
    html = "<p>" + tokens[0]
    for token, score in zip(tokens[1:], scores.squeeze().tolist()):
        color_value = 255 * score 
        html += f"<span style='background-color: rgb(255, {255-color_value}, {255-color_value}); color: black;'>{token}</span>"
    html += "</p>"
    return html

tokens = [tokenizer.decode([tok], clean_up_tokenization_spaces=False) for tok in encoding.input_ids.squeeze().tolist()]
html_output = generate_html(tokens, normalized_ppl)

display(HTML(html_output))


In [ ]:
performer_probs = softmax_fn(performer_logits).view(-1, V).to("cpu")
observer_scores = observer_logits.view(-1, V).to("cpu")

xppl = loss_fn(observer_scores[:-1], performer_probs[:-1]).view(-1, S - 1).to("cpu").float()
    
xppl, xppl.sum(1)


In [ ]:
normalized_xppl = xppl / torch.max(xppl)

display(HTML(html_output))

html_output = generate_html(tokens, normalized_xppl)
display(HTML(html_output))

binocular_score = normalized_ppl / normalized_xppl
normalized_binocular_score = binocular_score / torch.max(binocular_score)

html_output = generate_html(tokens, normalized_binocular_score)
display(HTML(html_output))


In [ ]:
plt.scatter(xppl.float(), ppl.float())
plt.title("Cross-Perplexity vs Perplexity")
plt.xlabel("Cross-Perplexity")
plt.ylabel("Perplexity")
plt.xlim(0, 12)
plt.ylim(0, 12)
plt.show()


In [ ]:
human = '''The healthcare industry typically draws sufficient attention to patients' education, especially when it comes to representatives of minority groups. That is why the article by McCurley et al. (2017) offers valuable information. The researchers demonstrate that Hispanic individuals deal with improved diabetes prevention when they participate in individual and group face-to-face sessions (McCurley et al., 2017). I believe that there is an apparent reason why such positive outcomes are achieved. It seems that face-to-face interventions are effective because patients have an opportunity to ask questions if they require explanations. Simultaneously, such educational sessions demonstrate that a patient is not unique with such a health issue. As a result, such interventions can improve people's morale, which, in turn, will lead to increased motivation to take preventive measures and protect health.'''

encoding = tokenize([human])

observer_logits, performer_logits = get_logits(encoding)

S = observer_logits.shape[-2]
V = observer_logits.shape[-1]


In [ ]:
shifted_logits = observer_logits[..., :-1, :].contiguous()
shifted_labels = encoding.input_ids[..., 1:].contiguous()

ppl = loss_fn(shifted_logits.transpose(1, 2).to("cpu"), shifted_labels).float()

normalized_ppl = ppl / torch.max(ppl)

tokens = [tokenizer.decode([tok], clean_up_tokenization_spaces=False) for tok in encoding.input_ids.squeeze().tolist()]
html_output = generate_html(tokens, normalized_ppl)

display(HTML(html_output))

performer_probs = softmax_fn(performer_logits).view(-1, V).to("cpu")
observer_scores = observer_logits.view(-1, V).to("cpu")

xppl = loss_fn(observer_scores[:-1], performer_probs[:-1]).view(-1, S - 1).to("cpu").float()
normalized_xppl = xppl / torch.max(xppl)

html_output = generate_html(tokens, normalized_xppl)
display(HTML(html_output))

binocular_score = normalized_ppl / normalized_xppl
normalized_binocular_score = binocular_score / torch.max(binocular_score)

html_output = generate_html(tokens, normalized_binocular_score)
display(HTML(html_output))


In [ ]:
plt.scatter(xppl.float(), ppl.float())
plt.title("Cross-Perplexity vs Perplexity")
plt.xlabel("Cross-Perplexity")
plt.ylabel("Perplexity")
plt.xlim(0, 12)
plt.ylim(0, 12)
plt.show()

ppl.sum(1) / xppl.sum(1)
